# Алгоритмы интеллектуальной обработки больших объемов данных
## Домашнее задание №2 - Дерево решений


**Общая информация**

**Срок сдачи:** до 27 марта 2018, 06:00   
**Штраф за опоздание:** -2 балла после 06:00 27 марта, -4 балла после 06:00 3 апреля, -6 баллов после 06:00 10 апреля

При отправлении ДЗ указывайте фамилию в названии файла   


Присылать ДЗ необходимо в виде ссылки на свой github репозиторий в slack @alkhamush
Необходимо в slack создать таск в приватный чат:   
/todo Фамилия Имя *ссылка на гитхаб* @alkhamush   
Пример:   
/todo Ксения Стройкова https://github.com/stroykova/spheremailru/stroykova_hw2.ipynb @alkhamush   

Используйте данный Ipython Notebook при оформлении домашнего задания.

###### Задание 1 (2 баллов)
Разберитесь в коде MyDecisionTreeClassifier, который уже частично реализован. В комментариях, где написано "Что делает этот блок кода?", ответьте на этот вопрос. Допишите код там, где написано "Ваш код". Ваша реализация дерева должна работать по точности не хуже DecisionTreeClassifier из sklearn. Точность проверяется на wine и Speed Dating Data.

###### Задание 2 (2 балла)
Добиться скорости работы на fit сравнимой со sklearn wine и Speed Dating Data. 
Для этого используем numpy. 

###### Задание 3 (2 балла)
Продемонстрируйте умение работать с Pipeline на данных Speed Dating Data и DecisionTreeClassifier. Нужно в pipeline произвести все необходимые преобразования данных и в конце обучить модель. Задание реализуйте под пунктом Задание 3 (уже написано ниже)

###### Задание 4 (2 балла)
Добавьте функционал, который определяет значения feature importance. Выведите 10 главных фичей под пунктом Задание 4 (уже написано ниже) для MyDecisionTreeClassifier и DecisionTreeClassifier так, чтобы сразу были видны выводы и по MyDecisionTreeClassifier, и по DecisionTreeClassifier. Используем данные Speed Dating Data.

###### Задание 5 (2 балла)
С помощью GridSearchCV или RandomSearchCV подберите наиболее оптимальные параметры для случайного леса (Выберете 2-3 параметра). Используем данные Speed Dating Data. Задание реализуйте под пунктом Задание 5 (уже написано ниже)


**Штрафные баллы:**

1. Невыполнение PEP8 -1 балл
2. Отсутствие фамилии в имени скрипта (скрипт должен называться по аналогии со stroykova_hw2.ipynb) -1 балл
3. Все строчки должны быть выполнены. Нужно, чтобы output команды можно было увидеть уже в git'е. В противном случае -1 балл
4. При оформлении ДЗ нужно пользоваться данным файлом в качестве шаблона. Не нужно удалять и видоизменять написанный код и текст. В противном случае -1 балл

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.datasets import load_wine
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import KFold, train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

%matplotlib inline
%load_ext pycodestyle_magic

In [3]:
#%%pycodestyle


class MyDecisionTreeClassifier:
    NON_LEAF_TYPE = 0
    LEAF_TYPE = 1

    def __init__(self, min_samples_split=2,
                 max_depth=None, sufficient_share=1.0,
                 criterion='gini', max_features=None):
        self.tree = dict()
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.sufficient_share = sufficient_share
        self.num_class = -1
        self.feature_importances_ = None
        if criterion == 'gini':
            self.G_function = self.__gini
        elif criterion == 'entropy':
            self.G_function = self.__entropy
        elif criterion == 'misclass':
            self.G_function = self.__misclass
        else:
            print('invalid criterion name')
            raise

        if max_features == 'sqrt':
            self.get_feature_ids = self.__get_feature_ids_sqrt
        elif max_features == 'log2':
            self.get_feature_ids = self.__get_feature_ids_log2
        elif max_features is None:
            self.get_feature_ids = self.__get_feature_ids_N
        else:
            print('invalid max_features name')
            raise

    def __gini(self, l_c, l_s, r_c, r_s):
        l_s = l_s.astype('float')
        r_s = r_s.astype('float')
        l_w = l_s / (l_s + r_s)
        return (l_w * (1 - ((l_c / l_s)**2).sum(axis=1,
                                                keepdims=True)) + (1 - l_w)
                * (1 - ((r_c / r_s)**2).sum(axis=1, keepdims=True)))

    def __entropy(self, l_c, l_s, r_c, r_s):
        p_l = l_c / l_s
        p_r = r_c / r_s
        h_l = -(p_l * np.nan_to_num(np.log2(p_l))).sum(axis=1, keepdims=True)
        h_r = -(p_r * np.nan_to_num(np.log2(p_r))).sum(axis=1, keepdims=True)
        l_w = l_s / (l_s + r_s)
        return l_w * h_l + (1 - l_w) * h_r

    def __misclass(self, l_c, l_s, r_c, r_s):
        l_w = l_s / (l_s + r_s)

        return (l_w * (1 - np.max((l_c / l_s), axis=1, keepdims=True))
                + (1 - l_w) * (1 - np.max((r_c / r_s), axis=1, keepdims=True)))

    def __get_feature_ids_sqrt(self, n_feature):
        feature_ids = np.arange(n_feature)
        np.random.shuffle(feature_ids)
        return feature_ids[:int(np.sqrt(n_feature))]

    def __get_feature_ids_log2(self, n_feature):
        feature_ids = np.arange(n_feature)
        np.random.shuffle(feature_ids)
        return feature_ids[:int(np.log2(n_feature))]

    def __get_feature_ids_N(self, n_feature):
        return np.arange(n_feature)

    def __sort_samples(self, x, y):
        sorted_idx = x.argsort()
        return x[sorted_idx], y[sorted_idx]

    def __div_samples(self, x, y, feature_id, threshold):
        left_mask = x[:, feature_id] > threshold
        right_mask = ~left_mask
        return x[left_mask], x[right_mask], y[left_mask], y[right_mask]

    def __find_threshold(self, x, y):
        # Что делает этот блок кода?
        # Сортировка элементов по x, для последующего поиска
        # разбиения о непрерывному признаку
        sorted_x, sorted_y = self.__sort_samples(x, y)
        class_number = self.num_class  # количество уникальных классов

        # Что делает этот блок кода?
        # вырезает из y середину по минимальному количеству
        # элементов разбиения(min_samples_split\cut_size) и находит индекс
        # пороговых значений разбиения
        cut_size = int(self.min_samples_split / 2) - 1
        splitted_sorted_y = (sorted_y[cut_size:-cut_size]
                             if cut_size != 0 else sorted_y)
        r_border_ids = (np.where(splitted_sorted_y[:-1] !=
                                 splitted_sorted_y[1:])[0] + (cut_size + 1))

        if len(r_border_ids) == 0:
            return float('+inf'), None

        # Что делает этот блок кода?
        eq_el_count = r_border_ids - np.append([cut_size], r_border_ids[:-1])
        # осуществляет подсчет интервалов между пороговыми значениями
        one_hot_code = np.zeros((r_border_ids.shape[0], class_number))
        # осуществляет кодировку категориальных признаков в каждой левой части разбиения
        one_hot_code[np.arange(r_border_ids.shape[0]),
                     sorted_y[r_border_ids - 1]] = 1
        class_increments = one_hot_code * eq_el_count.reshape(-1, 1)
        # осуществляет подсчет количества классов в левой части разбиения
        class_increments[0] = (class_increments[0] +
                               np.bincount(sorted_y[:cut_size],
                                           minlength=class_number))

        # Что делает этот блок кода?
        l_class_count = np.cumsum(class_increments, axis=0)
        # осуществляет подсчет общего количества классов слева
        r_class_count = np.bincount(sorted_y,
                                    minlength=class_number) - l_class_count
        # осуществляет подсчет общего количества классов справа
        l_sizes = r_border_ids.reshape(l_class_count.shape[0], 1)
        r_sizes = sorted_y.shape[0] - l_sizes

        # Что делает этот блок кода?
        gs = self.G_function(l_class_count, l_sizes, r_class_count, r_sizes)
        # осуществляет подсчет неопределенности по данному критерию
        idx = np.argmin(gs)  # и находит индекс наименьшей 

        # Что делает этот блок кода?
        left_el_id = l_sizes[idx][0]  # индекс налучшего разбиения
        return gs[idx], (sorted_x[left_el_id-1] + sorted_x[left_el_id]) / 2.0
        # возвращает значение наименьшей неопределенности и
        # медиану разбиения к ней

    def __create_leaf(self, y):
        classes_count = np.bincount(y,
                                    minlength=self.num_class).astype('float')
        probs = classes_count / np.sum(classes_count)
        return (self.LEAF_TYPE, classes_count.argmax(), probs)

    def __fit_node(self, x, y, node_id, depth, pred_f=-1):
        # Ваш код
        if (depth == self.max_depth or np.unique(y).size == 1 or y.size
                <= self.min_samples_split):
            print("Stopping criteria 1")
            print("Y: ", y)
            self.tree[node_id] = self.__create_leaf(y)
            return
        feature_ids = self.get_feature_ids(x.shape[1])
        thresholds = np.empty(x.shape[1])
        gss = np.empty(x.shape[1])
        for feature_id in feature_ids:
            gss[feature_id], thresholds[
                feature_id] = self.__find_threshold(x[:, feature_id], y)

        best_feature_id = gss.argmin()
        best_threashold = thresholds[best_feature_id]
        print("BEST GS: ", gss.min())
        print("BEST threshold: ", best_threashold)
        print("BEST FEATURE_ID: ", best_feature_id)
        l_x, r_x, l_y, r_y = self.__div_samples(x,
                                                y, best_feature_id,
                                                best_threashold)

        print("SPLIT")
        print("l_y: ", l_y)
        print("r_y: ", r_y)
        if l_y.size == 0 or r_y.size == 0:
            print("Stopping criteria 2")
            self.tree[node_id] = self.__create_leaf(y)
            return
        print(self.info_gain(y, l_y, r_y))
        self.feature_importances_[
            best_feature_id] = self.info_gain(y, l_y, r_y).max()
        self.tree[node_id] = (self.NON_LEAF_TYPE,
                              best_feature_id, best_threashold)
        # Left subtree
        self.__fit_node(l_x, l_y, node_id * 2 + 1, depth + 1)
        # Right subtree
        self.__fit_node(r_x, r_y, node_id * 2 + 2, depth + 1)

    def info_gain(self, y, l_y, r_y):
        y_c = np.bincount(y, minlength=self.num_class)
        l_y_c = np.bincount(l_y, minlength=self.num_class)
        r_y_c = np.bincount(r_y, minlength=self.num_class)
        p = l_y.size / (l_y.size + r_y.size)
        return ((1 - ((y_c / y.size)**2).sum()) -
                p * (1 - ((l_y_c / l_y.size)**2).sum()) -
                (1 - p) * (1 - ((r_y_c / r_y.size)**2).sum()))

    def fit(self, x, y):
        self.num_class = np.unique(y).size
        self.feature_importances_ = np.empty(x.shape[1])
        self.__fit_node(x, y, 0, 0)

    def __predict_class(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_class(x, 2 * node_id + 1)
            else:
                return self.__predict_class(x, 2 * node_id + 2)
        else:
            return node[1]

    def __predict_probs(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_probs(x, 2 * node_id + 1)
            else:
                return self.__predict_probs(x, 2 * node_id + 2)
        else:
            return node[2]

    def predict(self, X):
        return np.array([self.__predict_class(x, 0) for x in X])

    def predict_probs(self, X):
        return np.array([self.__predict_probs(x, 0) for x in X])

    def fit_predict(self, x_train, y_train, predicted_x):
        self.fit(x_train, y_train)
        return self.predict(predicted_x)

In [4]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

In [5]:
wine = load_wine()
X_train, X_test, y_train, y_test = train_test_split(wine.data, wine.target, test_size=0.1, stratify=wine.target)

## Проверка скорости работы на wine

In [7]:
%time clf.fit(X_train, y_train)

Wall time: 3 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [8]:
%time my_clf.fit(X_train, y_train)

BEST GS:  0.397398892431616
BEST threshold:  755.0
BEST FEATURE_ID:  12
SPLIT
l_y:  [1 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 1 0 2 0 0 0 0 0 0
 0 0 0 0 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2]
r_y:  [1 1 1 1 1 2 1 1 1 1 1 1 2 1 1 1 2 1 1 2 1 2 1 1 2 2 1 1 1 1 1 1 2 2 2 2 1
 1 2 1 1 1 1 1 2 1 2 2 2 1 1 2 1 1 1 1 0 2 1 2 0 1 2 1 1 1 1 2 1 1 1 2 2 1
 2 2 1 1 1 2 2 2 1 1 1 1 1 2 2 1 1 2 2 1 2 2 2 1 1 2 2]
0.2606479825683839
BEST GS:  0.05084745762711865
BEST threshold:  2.165
BEST FEATURE_ID:  6
SPLIT
l_y:  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
r_y:  [1 2 2 1 2 2 2 2]
0.19046251077276644
Stopping criteria 1
Y:  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
BEST GS:  0.0
BEST threshold:  2.085
BEST FEATURE_ID:  1
SPLIT
l_y:  [2 2 2 2 2 2]
r_y:  [1 1]
0.375
Stopping criteria 1
Y:  [2 2 2 2 2 2]
Stopping criteria 1
Y:  [1 1]
BEST GS:  0.16194611264405123
BE

## Проверка качества работы на wine

In [10]:
#f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')
print("Sklearn tree f1 on train: ", f1_score(y_pred=clf.predict(X_train), y_true=y_train, average='macro'))
print("Sklearn tree f1 on test: ", f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro'))

Sklearn tree f1 on train:  1.0
Sklearn tree f1 on test:  0.8311688311688311


In [11]:
#f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')
print("My tree f1 on train: ", f1_score(y_pred=my_clf.predict(X_train), y_true=y_train, average='macro'))
print("My tree f1 on test: ", f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro'))

My tree f1 on train:  1.0
My tree f1 on test:  0.8968253968253967


## Подготовка данных Speed Dating Data 

In [12]:
# тут делаете то же самое, что и на семинаре https://github.com/stroykova/spheremailru/blob/master/2018-02/lecture_04_trees/pract-speed-dating-trees-proc.ipynb
import pandas as pd
import numpy as np

class Transform_Speed_Dating(TransformerMixin):
    def __init__(self):
        self.data = None
    
    def fit(self, X):
        self.df = X
        return self
    
    def transform(self, X):
        df = self.df
        df = df.iloc[:, :97]
        df.iid.nunique()

        df = df.drop(['id'], axis=1)
        df = df.drop(['idg'], axis=1)
        df.drop_duplicates(subset=['iid']).gender.value_counts()
        df.drop_duplicates(subset=['iid']).condtn.value_counts()
        df = df.drop(['condtn'], axis=1)

        df = df.drop(['round'], axis=1)
        df = df.drop(['position', 'positin1'], axis=1)
        df = df.drop(['order'], axis=1)
        df = df.drop(['partner'], axis=1)
        df = df.drop(['age_o', 'race_o', 'pf_o_att',
                      'pf_o_sin', 'pf_o_int',
                      'pf_o_fun', 'pf_o_amb', 'pf_o_sha',
                      'dec_o', 'attr_o', 'sinc_o', 'intel_o', 'fun_o',
                      'amb_o', 'shar_o', 'like_o', 'prob_o', 'met_o'],
                     axis=1)
        df.drop_duplicates(subset=['iid']).age
        df.drop_duplicates('iid').age.isnull().sum()
        df = df.dropna(subset=['age'])

        df.field_cd.isnull().sum()

        df.loc[:, 'field_cd'] = df.loc[:, 'field_cd'].fillna(19)

        df = df.drop(['field'], axis=1)
        df = df.drop(['undergra'], axis=1)

        df.loc[:, 'mn_sat'] = df.loc[:, 'mn_sat'].str.replace(',', '').astype(np.float)

        df.drop_duplicates('iid').mn_sat
        df.drop_duplicates('iid').mn_sat.isnull().sum()
        df.loc[:, 'mn_sat'] = df.mn_sat.fillna(-999)

        df.loc[:, 'tuition'] = df.loc[:, 'tuition'].str.replace(',', '').astype(np.float)
        df.drop_duplicates('iid').tuition
        df.drop_duplicates('iid').tuition.isnull().sum()
        df.loc[:, 'tuition'] = df.tuition.fillna(-999)

        df.drop_duplicates('iid').race.value_counts()
        df.drop_duplicates('iid').age.isnull().sum()
        df.drop_duplicates('iid').race
        df.drop_duplicates('iid').imprace.isnull().sum()
        df.drop_duplicates('iid').imprelig.isnull().sum()

        df = df.dropna(subset=['imprelig', 'imprace'])

        df = df.drop(['from', 'zipcode'], axis=1)
        df.loc[:, 'income'] = df.loc[:, 'income'].str.replace(',', '').astype(np.float)

        df.drop_duplicates('iid').loc[:, 'income']

        df.loc[:, 'income'] = df.loc[:, 'income'].fillna(-999)

        df = df.dropna(subset=['date'])

        df.loc[:, 'career_c'] = df.loc[:, 'career_c'].fillna(18)

        df = df.drop(['career'], axis=1)
        df.loc[:, ['sports', 'tvsports', 'exercise', 'dining', 'museums', 'art', 'hiking', 'gaming',
                   'clubbing', 'reading', 'tv', 'theater', 'movies', 'concerts', 'music', 'shopping', 'yoga']
        ].isnull().sum()

        df = df.drop(['sports', 'tvsports', 'exercise', 'dining', 'museums', 'art', 'hiking', 'gaming',
                      'clubbing', 'reading', 'tv', 'theater', 'movies', 'concerts', 'music', 'shopping', 'yoga'], axis=1)
        df.drop_duplicates('iid').exphappy.isnull().sum()
        df.drop_duplicates('iid').expnum.isnull().sum()
        df = df.drop(['expnum'], axis=1)

        feat = ['iid', 'wave', 'attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1', 'shar1_1']
        temp = df.drop_duplicates(subset=['iid', 'wave']).loc[:, feat]
        temp.loc[:, 'totalsum'] = temp.iloc[:, 2:].sum(axis=1)
        idx = ((temp.wave < 6) | (temp.wave > 9)) & (temp.totalsum < 99)
        temp.loc[idx,]
        idx = ((temp.wave >= 6) & (temp.wave <= 9))

        temp.loc[idx,]
        df.loc[:, 'temp_totalsum'] = df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1', 'shar1_1']].sum(axis=1)
        df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1', 'shar1_1']] = \
            (df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1', 'shar1_1']].T / df.loc[:,
                                                                                              'temp_totalsum'].T).T * 100
        feat = ['iid', 'wave', 'attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1', 'amb2_1', 'shar2_1']

        temp = df.drop_duplicates(subset=['iid', 'wave']).loc[:, feat]
        temp.loc[:, 'totalsum'] = temp.iloc[:, 2:].sum(axis=1)

        idx = ((temp.wave < 6) | (temp.wave > 9)) & (temp.totalsum < 90) & (temp.totalsum != 0)
        temp.loc[idx,]

        idx = ((temp.wave >= 6) & (temp.wave <= 9))
        temp.loc[idx,]

        df.loc[:, 'temp_totalsum'] = df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1', 'amb2_1', 'shar2_1']].sum(axis=1)
        df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1', 'amb2_1', 'shar2_1']] = \
            (df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1', 'amb2_1', 'shar2_1']].T / df.loc[:,
                                                                                              'temp_totalsum'].T).T * 100
        df = df.drop(['temp_totalsum'], axis=1)
        for i in [4, 5]:
            feat = ['attr{}_1'.format(i), 'sinc{}_1'.format(i),
                    'intel{}_1'.format(i), 'fun{}_1'.format(i),
                    'amb{}_1'.format(i), 'shar{}_1'.format(i)]

            if i != 4:
                feat.remove('shar{}_1'.format(i))

            df = df.drop(feat, axis=1)

        df = df.drop(['wave'], axis=1)
        df_male = df.query('gender == 1').drop_duplicates(subset=['iid', 'pid']) \
            .drop(['gender'], axis=1) \
            .dropna()
        df_female = df.query('gender == 0').drop_duplicates(subset=['iid']) \
            .drop(['gender', 'match', 'int_corr', 'samerace'], axis=1) \
            .dropna()
        df_female.columns = df_female.columns + '_f'

        df_pair = df_male.join(df_female.set_index('iid_f'), on='pid', how='inner')
        df_pair = df_pair.drop(['iid', 'pid'], axis=1)
        return df_pair


# Data prep with pipeline
speed_dating_data = pd.read_csv('Speed Dating Data.csv', encoding='cp1251')
pipeline = Pipeline([('transform_speed_dating', Transform_Speed_Dating())])
data = pipeline.fit_transform(speed_dating_data)

X = data.iloc[:, 1:].values
y = data.iloc[:, 0].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

## Проверка скорости работы на Speed Dating Data 

In [16]:
# тут должен быть код типа f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')
%time clf.fit(X_train, y_train)

Wall time: 139 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [18]:
# тут должен быть код типа f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')
%time my_clf.fit(X_train, y_train)

BEST GS:  0.2685662135425549
BEST threshold:  4.0
BEST FEATURE_ID:  11
SPLIT
l_y:  [0 0 1 ... 0 1 0]
r_y:  [0 0 1 ... 1 0 1]
0.0023025335218598564
BEST GS:  0.2297110275691649
BEST threshold:  2.0
BEST FEATURE_ID:  6
SPLIT
l_y:  [0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0
 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 1 0 0 0 0 0
 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 1 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 

BEST GS:  0.09157392686804433
BEST threshold:  1.0
BEST FEATURE_ID:  34
SPLIT
l_y:  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1
 0 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
r_y:  [1 0 0 0 1 0 0 0]
0.0032668999246098784
BEST GS:  0.08536815014751117
BEST threshold:  9.0
BEST FEATURE_ID:  8
SPLIT
l_y:  []
r_y:  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1
 0 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Stopping criteria 2
BEST GS:  0.16666666666666666


BEST GS:  0.0
BEST threshold:  -0.22
BEST FEATURE_ID:  0
SPLIT
l_y:  [1 1]
r_y:  [0]
0.4444444444444444
Stopping criteria 1
Y:  [1 1]
Stopping criteria 1
Y:  [0]
BEST GS:  0.08281573498964806
BEST threshold:  1.0
BEST FEATURE_ID:  38
SPLIT
l_y:  [0 0 0 0 0 1 0 0 0 0 0 0 0]
r_y:  [0 0 0 0 1 0 0 1 0 0]
0.007445106877999236
BEST GS:  0.0
BEST threshold:  0.44999999999999996
BEST FEATURE_ID:  0
SPLIT
l_y:  [1]
r_y:  [0 0 0 0 0 0 0 0 0 0 0 0]
0.14201183431952646
Stopping criteria 1
Y:  [1]
Stopping criteria 1
Y:  [0 0 0 0 0 0 0 0 0 0 0 0]
BEST GS:  0.0
BEST threshold:  7.5
BEST FEATURE_ID:  22
SPLIT
l_y:  [0 0 0 0 0 0 0 0]
r_y:  [1 1]
0.31999999999999984
Stopping criteria 1
Y:  [0 0 0 0 0 0 0 0]
Stopping criteria 1
Y:  [1 1]
BEST GS:  0.2538740529370409
BEST threshold:  397.0
BEST FEATURE_ID:  32
SPLIT
l_y:  [0 0 0 0 0 0 0 0 0 0 0 1 1 0 1 0 0 0 1 1 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0
 0 0 0 1 1 0 1 1 0 0 0 0 0 1 0 0]
r_y:  [0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0

BEST GS:  0.08888888888888886
BEST threshold:  8.0
BEST FEATURE_ID:  38
SPLIT
l_y:  []
r_y:  [0 0 0 0 1 0 0 0 0 0 0 1 0 0 0]
Stopping criteria 2
BEST GS:  0.25925925925925924
BEST threshold:  10.0
BEST FEATURE_ID:  46
SPLIT
l_y:  [0 0 0 1 0 0 1]
r_y:  [1 1 0 1 0]
0.048015873015872956
BEST GS:  0.0
BEST threshold:  19.090909090909093
BEST FEATURE_ID:  49
SPLIT
l_y:  [1 1]
r_y:  [0 0 0 0 0]
0.40816326530612246
Stopping criteria 1
Y:  [1 1]
Stopping criteria 1
Y:  [0 0 0 0 0]
BEST GS:  0.0
BEST threshold:  -999.0
BEST FEATURE_ID:  4
SPLIT
l_y:  [0]
r_y:  [1 1 1 0]
0.17999999999999994
Stopping criteria 1
Y:  [0]
BEST GS:  0.0
BEST threshold:  0.845
BEST FEATURE_ID:  0
SPLIT
l_y:  [0]
r_y:  [1 1 1]
0.375
Stopping criteria 1
Y:  [0]
Stopping criteria 1
Y:  [1 1 1]
BEST GS:  0.304455457964182
BEST threshold:  28.0
BEST FEATURE_ID:  2
SPLIT
l_y:  [0 0 0 0 1 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0
 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1

SPLIT
l_y:  []
r_y:  [0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0]
Stopping criteria 2
Stopping criteria 1
Y:  [1]
BEST GS:  0.16098484848484845
BEST threshold:  20.0
BEST FEATURE_ID:  55
SPLIT
l_y:  [1 1 1 1]
r_y:  [1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
0.23507805325987147
Stopping criteria 1
Y:  [1 1 1 1]
BEST GS:  0.0
BEST threshold:  52805.0
BEST FEATURE_ID:  40
SPLIT
l_y:  [1]
r_y:  [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
0.09295570079883812
Stopping criteria 1
Y:  [1]
BEST GS:  0.0
BEST threshold:  -999.0
BEST FEATURE_ID:  4
SPLIT
l_y:  [0 0 0 0 0]
r_y:  [1 0 0 0 0 0 0 0 0 0 0 0]
0.0028835063437138014
Stopping criteria 1
Y:  [0 0 0 0 0]
BEST GS:  0.0
BEST threshold:  -999.0
BEST FEATURE_ID:  4
SPLIT
l_y:  []
r_y:  [1 0 0 0 0 0 0 0 0 0 0 0]
Stopping criteria 2
BEST GS:  0.3344220707150786
BEST threshold:  21.0
BEST FEATURE_ID:  2
SPLIT
l_y:  [0 1 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 1 0 0 1 0 0 0
 0 0 0 0 0 1 1 1 0 1 0 0 0 1 0 0 0

BEST GS:  0.2578468503765617
BEST threshold:  17.27922207779222
BEST FEATURE_ID:  15
SPLIT
l_y:  [1 1 0 1 0 1 0 0 1 0 1 0 0 0 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1]
r_y:  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
0.22353203543445718
BEST GS:  0.28035190615835776
BEST threshold:  0.28500000000000003
BEST FEATURE_ID:  0
SPLIT
l_y:  [1 1 0 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1]
r_y:  [1 0 0 0 0 0 0 1 1 0 1]
0.13171885346703233
BEST GS:  0.0947368421052633
BEST threshold:  5.0
BEST FEATURE_ID:  41
SPLIT
l_y:  []
r_y:  [1 1 0 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1]
Stopping criteria 2
BEST GS:  0.1590909090909091
BEST threshold:  2.0
BEST FEATURE_ID:  12
SPLIT
l_y:  []
r_y:  [1 0 0 0 0 0 0 1 1 0 1]
Stopping criteria 2
BEST GS:  0.0
BEST threshold:  23.75
BEST FEATURE_ID:  49
SPLIT
l_y:  [1]
r_y:  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
0.07396449704142016
Stopping criteria 1
Y:  [1]
Stopping criteria 1
Y:  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Wall time: 3.37 s


## Проверка качества работы на Speed Dating Data

In [19]:
# тут должен быть код типа %time clf.fit(X_train, y_train)
print(f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro'))

0.5493550934456436


In [21]:
# тут должен быть код типа %time my_clf.fit(X_train, y_train)
print(f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro'))

0.5132666445424232


## Задание 3

In [22]:
pipeline = Pipeline([('clf', MyDecisionTreeClassifier())])
X = pipeline.fit(X_train, y_train)
print(f1_score(y_pred=pipeline.predict(X_test), y_true=y_test, average='macro'))

BEST GS:  0.2685662135425549
BEST threshold:  4.0
BEST FEATURE_ID:  11
SPLIT
l_y:  [0 0 1 ... 0 1 0]
r_y:  [0 0 1 ... 1 0 1]
0.0023025335218598564
BEST GS:  0.2297110275691649
BEST threshold:  2.0
BEST FEATURE_ID:  6
SPLIT
l_y:  [0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0
 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 1 0 0 0 0 0
 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 1 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 

BEST GS:  0.0
BEST threshold:  8.555555555555555
BEST FEATURE_ID:  25
SPLIT
l_y:  [0 0 0 0 0 0 0]
r_y:  [1 1]
0.345679012345679
Stopping criteria 1
Y:  [0 0 0 0 0 0 0]
Stopping criteria 1
Y:  [1 1]
BEST GS:  0.1904761904761905
BEST threshold:  10.0
BEST FEATURE_ID:  56
SPLIT
l_y:  [1 0 0]
r_y:  [0 0 0 0 0 0 0 1 1 1 0 1]
-5.551115123125783e-17
BEST GS:  0.0
BEST threshold:  0.0
BEST FEATURE_ID:  1
SPLIT
l_y:  []
r_y:  [1 0 0]
Stopping criteria 2
BEST GS:  0.0
BEST threshold:  10.0
BEST FEATURE_ID:  56
SPLIT
l_y:  []
r_y:  [0 0 0 0 0 0 0 1 1 1 0 1]
Stopping criteria 2
BEST GS:  0.17134172818063403
BEST threshold:  10.0
BEST FEATURE_ID:  62
SPLIT
l_y:  []
r_y:  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 1 1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

 0 1 0 1 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 1 0]
0.002452006811280688
BEST GS:  0.29759862778730695
BEST threshold:  9162.0
BEST FEATURE_ID:  36
SPLIT
l_y:  [0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0
 1 0 0 0 0 0]
r_y:  [0 1 1 1 0 1 1 0 1 0]
0.05852285428067583
BEST GS:  0.20419739081111735
BEST threshold:  15.0
BEST FEATURE_ID:  56
SPLIT
l_y:  []
r_y:  [0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0
 1 0 0 0 0 0]
Stopping criteria 2
BEST GS:  0.15999999999999992
BEST threshold:  27.0
BEST FEATURE_ID:  2
SPLIT
l_y:  [1 1 1 1]
r_y:  [0 0 1 0 1 0]
0.21333333333333332
Stopping criteria 1
Y:  [1 1 1 1]
BEST GS:  0.2222222222222222
BEST threshold:  0.43000000000000005
BEST FEATURE_ID:  0
SPLIT
l_y:  [1 0 1]
r_y:  [0 0 0]
0.2222222222222222
BEST GS:  0.0
BEST threshold:  25.0
BEST FEATURE_ID:  2
SPLIT
l_y:  [1 1]
r_y:  [0]
0.4444444444444444
Stopping criteria 1
Y:  [1 1]
Stopping criteria 1
Y:  [0]
Stopping criteria 1
Y:  [0 0 0]
BEST GS:

BEST GS:  0.2871732026143791
BEST threshold:  10.0
BEST FEATURE_ID:  29
SPLIT
l_y:  []
r_y:  [1 1 1 1 0 0 1 1 1 0 1 0 0 1 0 0 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 0 0]
Stopping criteria 2
BEST GS:  0.28924162257495595
BEST threshold:  7.0
BEST FEATURE_ID:  62
SPLIT
l_y:  [0 0 0 0 1 0 0 0 0 0 0 1 0 0 0]
r_y:  [0 1 1 0 0 1 0 0 0 1 1 0]
0.03964334705075462
BEST GS:  0.08888888888888886
BEST threshold:  8.0
BEST FEATURE_ID:  38
SPLIT
l_y:  []
r_y:  [0 0 0 0 1 0 0 0 0 0 0 1 0 0 0]
Stopping criteria 2
BEST GS:  0.25925925925925924
BEST threshold:  10.0
BEST FEATURE_ID:  46
SPLIT
l_y:  [0 0 0 1 0 0 1]
r_y:  [1 1 0 1 0]
0.048015873015872956
BEST GS:  0.0
BEST threshold:  19.090909090909093
BEST FEATURE_ID:  49
SPLIT
l_y:  [1 1]
r_y:  [0 0 0 0 0]
0.40816326530612246
Stopping criteria 1
Y:  [1 1]
Stopping criteria 1
Y:  [0 0 0 0 0]
BEST GS:  0.0
BEST threshold:  -999.0
BEST FEATURE_ID:  4
SPLIT
l_y:  [0]
r_y:  [1 1 1 0]
0.17999999999999994
Stopping criteria 1
Y:  [0]
BEST GS:  0.0
BEST threshold:  0.

BEST GS:  0.11015258921062134
BEST threshold:  0.0
BEST FEATURE_ID:  23
SPLIT
l_y:  [0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0
 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0
 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0
 0 1 0 0 0 0 0 0 0 0 0]
r_y:  [0 0 0 0 0 0 1 0 1]
0.002176007424583999
BEST GS:  0.10355644848613008
BEST threshold:  20.0
BEST FEATURE_ID:  26
SPLIT
l_y:  []
r_y:  [0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0
 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0
 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0
 

BEST GS:  0.0
BEST threshold:  2.0
BEST FEATURE_ID:  44
SPLIT
l_y:  [1 1 1 1 1]
r_y:  [0 0 1 1]
0.12345679012345678
Stopping criteria 1
Y:  [1 1 1 1 1]
BEST GS:  0.0
BEST threshold:  28.0
BEST FEATURE_ID:  2
SPLIT
l_y:  []
r_y:  [0 0 1 1]
Stopping criteria 2
BEST GS:  0.10155610155610152
BEST threshold:  -999.0
BEST FEATURE_ID:  40
SPLIT
l_y:  [0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
r_y:  [0 0 0 0 0 0 0 0 1 0 0 1 1 0 0 1]
0.011756930675849708
BEST GS:  0.06349206349206352
BEST threshold:  18.333333333333332
BEST FEATURE_ID:  51
SPLIT
l_y:  [1 0 1]
r_y:  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
0.10884353741496589
BEST GS:  0.0
BEST threshold:  0.34
BEST FEATURE_ID:  0
SPLIT
l_y:  []
r_y:  [1 0 1]
Stopping criteria 2
Stopping criteria 1
Y:  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
BEST GS:  0.09999999999999995
BEST threshold:  0.05500000000000001
BEST FEATURE_ID:  0
SPLIT
l_y:  [0 0 0 0 0 0 0 0 0 0 0]
r_y:  [1 0 1 1 1]
0.275
Stopping criteria 1
Y:  [0 0 0 0 0 0 0 0 0 0 0]
BEST GS:  0.0
BE

## Задание 4

In [23]:
feature_ids_sklearn = clf.feature_importances_.argsort()[:10]
print("Feature importance")
print(data.columns[1:][feature_ids_sklearn].values.reshape(-1, 1))
feature_ids_my_clf = my_clf.feature_importances_.argsort()[:10]
print("My clf Feature importance")
print(data.columns[1:][feature_ids_my_clf].values.reshape(-1, 1))

Sklearn Feature importance
[['samerace']
 ['intel3_1_f']
 ['mn_sat']
 ['amb1_1']
 ['pid_f']
 ['intel3_1']
 ['race_f']
 ['race']
 ['imprelig_f']
 ['goal']]
My clf Feature importance
[['amb2_1_f']
 ['amb2_1']
 ['sinc3_1_f']
 ['shar2_1']
 ['go_out']
 ['intel2_1']
 ['pid_f']
 ['mn_sat']
 ['go_out_f']
 ['attr3_1']]


## Задание 5

In [24]:
clf = RandomForestClassifier()
params = {"max_depth": list(range(1, 10)) + [None], "criterion": ["gini", "entropy"]}

grid = GridSearchCV(clf, params, cv=2, n_jobs=-1, verbose=True)

grid.fit(X_train, y_train)
print("Best score")
print(grid.best_score_)
print("Best estimator")
print(grid.best_estimator_)

Fitting 2 folds for each of 20 candidates, totalling 40 fits
Best estimator
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=9, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)
Best score
0.8391219783273132


[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:   14.4s finished
